In [ ]:
import pandas as pd
import tkinter as tk
import sys
import openpyxl 
from tkinter import filedialog, messagebox


def load_excel(file_path):
    """Carrega a planilha considerando a linha 17 como cabeçalho e ajustando o engine."""
    
    # Verifica se o arquivo é .xls ou .xlsx
    if file_path.endswith('.xls'):
        engine = 'xlrd'  # Para arquivos .xls (versão antiga do Excel)
    else:
        engine = 'openpyxl'  # Para arquivos .xlsx (versão moderna)

    # Carregar planilha com o engine adequado
    df = pd.read_excel(file_path, sheet_name=0, header=16, engine=engine)
    
    return df

def upload_and_compare_bom():
   
    # Criar janela para seleção de arquivos
    root = tk.Tk()
    root.withdraw()  # Esconde a janela principal

    # 🔹 Exibir aviso Antes da execução
    messagebox.showwarning("Aviso ⚠️", "Certifique-se de que os arquivos foram baixados com o filtro KINAXIS antes da comparação!\n \n Selecione os arquivos: \n -BOM princial (A)\n -BOM segundaria (B)")

    #messagebox.showinfo("Aviso", "Selecione os arquivos:\n 'BOM A'\n 'BOM B' ")
    new_bom_path = filedialog.askopenfilename(title="Selecione o arquivo NEW BOM", filetypes=[("Excel files", "*.xlsx *.xls")])
    df1=load_excel(new_bom_path)
    
    old_bom_path = filedialog.askopenfilename(title="Selecione o arquivo OLD BOM", filetypes=[("Excel files", "*.xlsx *.xls")])
    df2=load_excel(old_bom_path)
    return df1, df2

#################################### selecao e leitura de arquivos  ####################################################
try:
    df1,df2=upload_and_compare_bom()
except:
    messagebox.showwarning('Aviso ⚠️',"Erro: Você precisa selecionar ambos os arquivos.")
    sys.exit()
try:
    # Identificar linhas em df1 cujo 'Item Number' não aparece em df2
    NOT_find_DF1_DF2 = df1[~df1["Item Number"].isin(df2["Item Number"])]
    NOT_find_DF2_DF1 = df2[~df2["Item Number"].isin(df1["Item Number"])]

    # Selecionar apenas as colunas desejadas
    columns_to_keep = ["Position #", "Item Number", "Item Description",'Qty', "Ref Des"]
    NOT_find_DF1_DF2 = NOT_find_DF1_DF2[columns_to_keep]
    NOT_find_DF2_DF1 = NOT_find_DF2_DF1[columns_to_keep]
except:
    messagebox.showwarning('Aviso ⚠️',"Erro: leitura ou indices:\n - verifique os arquivos selecionados \n - verificar titulo das colunas (F,G,H,K,N)")
    sys.exit()

# Criar um objeto do ExcelWriter para salvar as informações
with pd.ExcelWriter("Comparacao_BOM.xlsx", engine="xlsxwriter") as writer:

    # Escrever os dados encontrados filtrados (In A not B)
    NOT_find_DF1_DF2.to_excel(writer, sheet_name="Sheet1", startrow=1, index=False)
    # Escrever os dados encontrados filtrados (In B not A)
    NOT_find_DF2_DF1.to_excel(writer, sheet_name="Sheet1", startrow=len(NOT_find_DF1_DF2)+4, index=False)

    # Definir a posição inicial para inserir os dados do loop
    start_row = len(NOT_find_DF1_DF2) + len(NOT_find_DF2_DF1) + 7

    # Percorrer os valores de 'Item Number' e adicionar os correspondentes de df2
    for item in df1['Item Number']:
        
        try:
            track_line = df2[df2["Item Number"] == item]
            # Pegar os valores de 'Ref Des' para os itens correspondentes
            ref_des_df1 = df1[df1["Item Number"] == item]["Ref Des"].values[0] if not df1[df1["Item Number"] == item]["Ref Des"].isna().all() else None
            ref_des_df2 = track_line["Ref Des"].values if not track_line["Ref Des"].isna().all() else None
            
            # Pegar os valores de 'Ref Des' para os itens correspondentes e dividir em listas
            ref_des_df1_a = str(df1[df1["Item Number"] == item]["Ref Des"].values[0]).split(",") if not df1[df1["Item Number"] == item]["Ref Des"].isna().all() else []
            ref_des_df2_b = list(track_line["Ref Des"].dropna().apply(lambda x: str(x).split(",")).explode()) if not track_line["Ref Des"].isna().all() else []

            # Identificar os valores que estão sobrando em df1 e faltando em df2
            sobrando_df1 = set(ref_des_df1_a) - set(ref_des_df2_b)
            sobrando_df2 = set(ref_des_df2_b) - set(ref_des_df1_a)
            # Verificar se 'Ref Des' está ausente em ambos os DataFrames

            if ref_des_df1 is None and ref_des_df2 is None:
                continue  # Se ambos forem NaN, pula para o próximo item

            if not track_line.empty:
                track_line_filtrado = track_line[track_line["Ref Des"] != ref_des_df1]
                track_line_filtrado_a = track_line[track_line["Ref Des"] != ",".join(ref_des_df1_a)]

                if not track_line_filtrado.empty:  # Evita salvar linhas vazias
                    track_line_filtrado = track_line_filtrado[columns_to_keep]  # Filtrar colunas
                    # Transformar listas em conjuntos para permitir a remoção
                    result_set = set(ref_des_df1_a) | set(sobrando_df1)  # União dos elementos
                    result_set -= set(sobrando_df2)  # Remove elementos de sobrando_df2

                    # Converter de volta para lista e unir com vírgulas
                    track_line_filtrado["New Ref Des"] = ",".join(sorted(result_set))  # `sorted()` para manter a ordem}
                    track_line_filtrado["adicionados"] = ",".join(sobrando_df1)  # Adiciona valores faltando
                    track_line_filtrado["removidos"] = ",".join(sobrando_df2)  # Adiciona valores extras
                    track_line_filtrado.to_excel(writer, sheet_name="Sheet1", startrow=start_row, index=False, header=False)
                    start_row += len(track_line_filtrado)  # Atualizar a posição da próxima linha
        except:
            messagebox.showwarning('Aviso ⚠️',"Erro: Ref Des Logical\n - verificar se Colunas(G,N) possuem valores inesperados \n - verificar se existe valores foras dos limites da BOM ")
            sys.exit()
                    
    # Ajuste de estilo da workbook e a worksheet ####################################################################################

    try:   
        workbook = writer.book
        worksheet = writer.sheets["Sheet1"]


        worksheet.set_column("B:B", 25)
        worksheet.set_column("C:C", 60)

        middle_align_format_G = workbook.add_format({
            "border": 1, # Adiciona borda padrão em todas as direções
            "align": "center",  # Centraliza horizontalmente
            "valign": "vcenter",  # Centraliza verticalmente 
            "bg_color": "#00FF00" # Formato para fundo verde
        })
        middle_align_format_R = workbook.add_format({
            "border": 1, # Adiciona borda padrão em todas as direções
            "align": "center",  # Centraliza horizontalmente
            "valign": "vcenter",  # Centraliza verticalmente 
            "bg_color": "#FF0000"  # Formato para fundo vermelho
        })
        title_format = workbook.add_format({
            "bold": True,  # Negrito
            "border": 1,   # Bordas em todas as direções
            "align": "center",  # Centralizado horizontalmente
            "valign": "vcenter",  # Centralizado verticalmente
        })
        
        # Criar formatação para wrap text e cores
        wrap_format = workbook.add_format({
            "text_wrap": True                                 
        })
        wrap_format_R = workbook.add_format({
            "text_wrap": True , 
            "bg_color": "#FF0000",  # Formato para fundo vermelho
            "border": 1 # Adiciona borda padrão em todas as direções                              
        })
        wrap_format_G = workbook.add_format({
            "text_wrap": True,
            "bg_color": "#00FF00", # Formato para fundo verde
            "border": 1 # Adiciona borda padrão em todas as direções                             
        })

        # Aplica formatação condicionalmente, sem apagar texto existente, Item Numbers add e remov
        worksheet.conditional_format(2, 1, len(NOT_find_DF1_DF2)+1, 1, 
        {"type": "no_blanks", "format": middle_align_format_G})  # Fundo verde

        worksheet.conditional_format(len(NOT_find_DF1_DF2)+5, 1, len(NOT_find_DF2_DF1)+len(NOT_find_DF1_DF2)+4, 1, 
        {"type": "no_blanks", "format": middle_align_format_R})  # Fundo vermelho

        # Aplica formatação condicionalmente, sem apagar texto existente, Ref Des add e remov
        worksheet.conditional_format(len(NOT_find_DF2_DF1)+len(NOT_find_DF1_DF2)+6, 6, start_row, 6, 
        {"type": "no_blanks", "format": wrap_format_G})  # Fundo verde

        worksheet.conditional_format(len(NOT_find_DF2_DF1)+len(NOT_find_DF1_DF2)+6, 7, start_row, 7, 
        {"type": "no_blanks", "format": wrap_format_R})  # Fundo vermelho

        # Ajustar largura e aplicar formatação na planilha
        worksheet.set_column("E:H", 30, wrap_format)
    except:
        messagebox.showwarning('Aviso ⚠️',"Erro: Limites ou formatação nova sheet (only code)")
        sys.exit()

    # Linhas adicionais #############################################################################################################

    try:
        # Valor de Qty somado em Item Number add
        worksheet.merge_range('A1:E1', ' Exist In A not B',  middle_align_format_G)
        num=len(NOT_find_DF1_DF2)+4
        text01=f"{'A'}{num}{':'}{'E'}{num}"
        worksheet.merge_range(text01, 'Exist In B not A',  middle_align_format_R)

        # Valor de Qty somado em Item Number add
        total_qty_df1 = NOT_find_DF1_DF2["Qty"].sum()
        num=len(NOT_find_DF1_DF2)+3
        text01=f"{'D'}{num}"
        worksheet.write(text01, total_qty_df1,  wrap_format_G)
        # Incrementa para achar a linha correta sem valor 
        num = num + len(NOT_find_DF2_DF1)+3
        # Valor de Qty somado em Item Number remov
        total_qty_df2 =  NOT_find_DF2_DF1["Qty"].sum()
        text02=f"{'D'}{num}"
        worksheet.write(text02, total_qty_df2,  wrap_format_R)
        

        # Linha que deve ser escrito o titulo
        numero=len(NOT_find_DF1_DF2) + len(NOT_find_DF2_DF1) + 7
        # Valores para escrita do titulo 
        texto=['A','B','C','D','E','F','G','H']
        titulo=["Position #","Item Number","Item Description",'Qty'," B Ref Des",'A Ref Des', "A → B Add Ref Des", "A → B Remov Ref Des "]
        for x , y in zip(texto, titulo):
            text=f"{x}{numero}"
            worksheet.write(text, y, title_format)
    except:
        messagebox.showwarning('Aviso ⚠️',"Erro: New sheet Index error (only code)\n - indices nova sheet\n - limites de referencia  ")
        sys.exit()
    
    messagebox.showinfo('Aviso',"Comparação concluída! O arquivo 'Comparacao_BOM.xlsx' foi gerado com os resultados.")


    


SystemExit: 

C:\Users\jagfisou\AppData\Roaming\Python\Python313\site-packages\IPython\core\interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
